<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/Architecture-Files/17_Dynamic_Constraint_Adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mount Google Drive to access the file
from google.colab import drive
drive.mount('/content/drive')  # Mount Google Drive

# Import necessary libraries
import pandas as pd

# Provide the path to the file in Google Drive
file_path = '/content/drive/My Drive/timeseries_data.csv.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the data
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Dependency Ratio,Unnamed: 11,Unnamed: 12,Median Age,Unnamed: 14,Unnamed: 15
0,Name,GENC,Year,Total Population,Growth Rate,Population Density (per sq km),Total Fertility Rate,Life Expectancy at Birth,Under-5 Mortality Rate,Sex Ratio of the Population,Youth and Old Age (0-14 and 65+),Youth (0-14),Old Age (65+),Both Sexes,Male,Female
1,-> 2024,NaN,NaN,--,--,--,--,--,--,--,--,--,--,--,--,--
2,Canada,CA,2024,"38,904,514",0.72,4.3,1.44,83.9,4.8,0.99,56.8,23.9,32.9,42.5,43.9,41.2
3,-> 2025,NaN,NaN,--,--,--,--,--,--,--,--,--,--,--,--,--
4,Canada,CA,2025,"39,187,155",0.73,4.3,1.43,84.8,4.4,0.99,57.7,23.8,33.9,42.8,44.1,41.4


In [3]:
# --- Extract and Read Data ---
# Extracting and reading the data (in this case, just displaying the first few rows)
print("Data Preview:")
print(df.head())

# --- Data Operations and Explanations ---
# Let's start by understanding the data structure and its columns
print("Data Columns:", df.columns)

# --- Explanation for Extracting Data ---
# 1. We are using the 'df.head()' function to extract the first few rows of the dataset for inspection.
#    This helps us confirm the format and contents of the data.
# 2. Pandas provides powerful data manipulation capabilities, making it easy to access and handle the dataset.
# 3. The 'df.columns' function helps us understand the structure of the data, ensuring that we can reference the columns accurately.

# --- Dynamic Constraint Adaptation: Resilient Constrained Learning ---
# In this section, we implement dynamic constraint adaptation during the learning process.
# Here, we adjust the error constraint based on the model's performance.

# Import necessary libraries for optimization and dynamic constraint adaptation
import numpy as np
from scipy.optimize import minimize

# Define the objective function (error function) for forecasting
def objective_function(params, X, y):
    predictions = X.dot(params)  # Linear model: y = X * params
    error = np.sum((predictions - y) ** 2)  # Mean Squared Error
    return error

# Define the constraint function that will adapt dynamically based on model performance
def dynamic_constraint_function(params, X, y, learning_progress):
    # Dynamic error constraint based on learning progress
    base_error_bound = 5  # Base error bound for early steps
    error_growth_factor = 1 + learning_progress  # Adjust error bound based on learning progress
    error_bound = base_error_bound * error_growth_factor  # Dynamic bound

    predictions = X.dot(params)
    errors = np.abs(predictions - y)

    # Return the difference between the error bound and max error, to ensure constraint is met
    return error_bound - np.max(errors)

# Prepare the data (X: input features, y: target values)
X = df.drop('target_column', axis=1).values  # Assuming 'target_column' is the column we want to predict
y = df['target_column'].values

# Initial guess for the parameters
initial_params = np.zeros(X.shape[1])

# Define the learning progress, starting at 0 (it will increase as we iterate)
learning_progress = 0.0

# Define the optimization problem with dynamic constraint adaptation
result = minimize(
    objective_function, initial_params, args=(X, y), constraints={'type': 'ineq', 'fun': dynamic_constraint_function, 'args': (X, y, learning_progress)},
    options={'disp': True}
)

# --- Dynamic Constraint Adaptation Results ---
print("Optimization Result with Dynamic Constraints:")
print(result)

# --- Plotting the results ---
import matplotlib.pyplot as plt

# Plot the original vs predicted values
y_pred = X.dot(result.x)
plt.plot(y, label='True Values', color='blue')
plt.plot(y_pred, label='Predicted Values', color='red')
plt.title("True vs Predicted Values (With Dynamic Constraints)")
plt.xlabel('Time Step')
plt.ylabel('Values')
plt.legend()
plt.show()

# --- Explanation of Dynamic Constraint Adaptation ---
# 1. Dynamic Constraint Adaptation adjusts the forecast error bounds during the model training process.
#    As the model progresses, the constraint becomes more flexible, allowing the model to focus on later steps where performance is more challenging.
# 2. The dynamic_constraint_function applies a variable error bound, which increases based on the learning progress.
#    This means that as the model learns and improves, it has more flexibility on earlier time steps but becomes stricter on later time steps when the model struggles.
# 3. The 'objective_function' minimizes the forecast error (Mean Squared Error), and the 'dynamic_constraint_function' ensures that error bounds are adjusted as per learning progress.
# 4. The 'minimize' function optimizes the model parameters based on this adaptive strategy, improving performance across all forecasting time steps.


Data Preview:
  Unnamed: 0 Unnamed: 1 Unnamed: 2        Unnamed: 3   Unnamed: 4  \
0       Name       GENC       Year  Total Population  Growth Rate   
1    -> 2024        NaN        NaN                --           --   
2     Canada         CA       2024        38,904,514         0.72   
3    -> 2025        NaN        NaN                --           --   
4     Canada         CA       2025        39,187,155         0.73   

                       Unnamed: 5            Unnamed: 6  \
0  Population Density (per sq km)  Total Fertility Rate   
1                              --                    --   
2                             4.3                  1.44   
3                              --                    --   
4                             4.3                  1.43   

                 Unnamed: 7              Unnamed: 8  \
0  Life Expectancy at Birth  Under-5 Mortality Rate   
1                        --                      --   
2                      83.9                     4.8

KeyError: "['target_column'] not found in axis"